# Import

In [1]:
import astropy.units as u
from astropy.coordinates.sky_coordinate import SkyCoord
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import os
import copy

from kesacco import clustpipe

# Parameters

### Define the simulation

In [2]:
cpipe = clustpipe.ClusterPipe(silent=False, output_dir='/Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev')

             _  __  ___    __    __     ___   ___   __         
            | |/ / | __| /' _/  /  \   / _/  / _/  /__\        
            |   <  | _|  `._`. | /\ | | \__ | \__ | \/ |       
            |_|\_\ |___| |___/ |_||_|  \__/  \__/  \__/        
Keen Event Simulation and Analysis for CTA Cluster Observations
---------------------------------------------------------------


### Define the cluster object

In [3]:
# Set the cluster basic properties
cpipe.cluster.name     = 'Perseus'
cpipe.cluster.redshift = 0.017284
cpipe.cluster.M500     = 6.2e14*u.solMass # Used for getting R500 and the pressure profile
cpipe.cluster.coord    = SkyCoord("3h19m47.2s +41d30m47s", frame='icrs')

# Truncate the cluster at 1.5 deg to allow small map field of view (for faster code) without loss of flux
cpipe.cluster.theta_truncation = 1.5*u.deg

# The target gas density [Churazov et al. 2003]
cpipe.cluster.density_gas_model = {'name':'doublebeta', 'beta1':1.2, 'r_c1':57*u.kpc, 'n_01':0.046*u.cm**-3,
                                   'beta2':0.71, 'r_c2':278*u.kpc, 'n_02':0.0036*u.cm**-3}

# The thermal profile (assuming Planck 2013 UPP)
cpipe.cluster.set_pressure_gas_gNFW_param('P13UPP')

# CR physics
cpipe.cluster.spectrum_crp_model = {'name':'PowerLaw', 'Index':2.2}
cpipe.cluster.set_density_crp_isobaric_scal_param(scal=1.0)
cpipe.cluster.X_crp_E = {'X':1.0, 'R_norm':cpipe.cluster.R500}

# Sampling
cpipe.cluster.map_reso = 0.01*u.deg      # Ideally should be few times smaller than the PSF
cpipe.cluster.Npt_per_decade_integ = 30

# Get information about the state of the cluster model
cpipe.cluster.print_param()

--- silent
    True
    <class 'bool'>
--- output_dir
    /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev
    <class 'str'>
--- cosmo
    FlatLambdaCDM(name="Planck15", H0=67.7 km / (Mpc s), Om0=0.307, Tcmb0=2.725 K, Neff=3.05, m_nu=[0.   0.   0.06] eV, Ob0=0.0486)
    <class 'astropy.cosmology.core.FlatLambdaCDM'>
--- name
    Perseus
    <class 'str'>
--- coord
    <SkyCoord (ICRS): (ra, dec) in deg
    (49.94666667, 41.51305556)>
    <class 'astropy.coordinates.sky_coordinate.SkyCoord'>
--- redshift
    0.017284
    <class 'float'>
--- D_ang
    74.8907830317466 Mpc
    <class 'astropy.units.quantity.Quantity'>
--- D_lum
    77.50198024167615 Mpc
    <class 'astropy.units.quantity.Quantity'>
--- M500
    620000000000000.0 solMass
    <class 'astropy.units.quantity.Quantity'>
--- R500
    1317.548394363056 kpc
    <class 'astropy.units.quantity.Quantity'>
--- theta500
    1.0080007077672122 deg
    <class 'astropy.units.quantity.Quantity'>
--- R_truncation
    1.9606361149510187 Mpc

### Define the point source object

In [4]:
name='IC310'
spatial={'type':'PointSource',
         'param':{'RA': {'value':SkyCoord("3h16m42.98s +41d19m30s").ra.to('deg'),  'free':False},
                  'DEC':{'value':SkyCoord("3h16m42.98s +41d19m30s").dec.to('deg'), 'free':False}}}
spectral={'type':'PowerLaw',
          'param':{'Prefactor':{'value':4.3e-12/u.cm**2/u.TeV/u.s, 'free':True},
                   'Index':{'value':-1.95, 'free':True},
                   'PivotEnergy':{'value':1.0*u.TeV, 'free':False}}}

cpipe.compact_source.add_source(name, spatial, spectral)# Add the source to the model

In [5]:
# Show the status of the compact sources in the sky model
cpipe.compact_source.print_source()

--- IC310
    -- Spatial model: PointSource
         RA: {'value': <Longitude 49.17908333 deg>, 'free': False}
         DEC: {'value': <Latitude 41.325 deg>, 'free': False}
    -- Spectral model: PowerLaw
         Prefactor: {'value': <Quantity 4.3e-12 1 / (cm2 s TeV)>, 'free': True}
         Index: {'value': -1.95, 'free': True}
         PivotEnergy: {'value': <Quantity 1. TeV>, 'free': False}
    -- Temporal model: Constant
         Normalization: {'value': 1.0, 'free': False}


### Define the observations

In [6]:
# One pointing offset +0 +0
offs = 1.0
cpipe.obs_setup.add_obs(obsid='001', name='Perseus_Ptg1', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value+0, cpipe.cluster.coord.dec.value+offs, unit='deg'),
                        rad=10*u.deg,
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-01T00:00:00.0', tmax='2020-01-01T00:30:00.0', deadc=0.95)
'''
# One pointing offset +sqrt(3)/2 -0.5
cpipe.obs_setup.add_obs(obsid='002', name='Perseus_Ptg2', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value+np.sqrt(3)/2, cpipe.cluster.coord.dec.value-0.5, unit='deg'),
                        rad=5*u.deg, 
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-01T01:00:00.0', tmax='2020-01-01T01:30:00.0', deadc=0.95)

# One pointing offset -sqrt(3)/2 -0.5
cpipe.obs_setup.add_obs(obsid='003', name='Perseus_Ptg3', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value-np.sqrt(3)/2, cpipe.cluster.coord.dec.value-0.5, unit='deg'),
                        rad=5*u.deg,
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-01T02:00:00.0', tmax='2020-01-01T02:30:00.0', deadc=0.95)
'''
# Print info
cpipe.obs_setup.print_obs()

=== Perseus_Ptg1, ObsID 001
        RA-Dec:    49.94666666666666, 42.51305555555555 deg
        GLON-GLAT: 150.0078784158764, -12.426877669678154 deg
        ROI rad: 10.0 deg
        tmin: 2020-01-01T00:00:00.0
        tmax: 2020-01-01T00:30:00.0
        emin: 0.05 TeV
        emax: 100.0 TeV
        deadc: 0.95
        caldb: prod3b-v2
        irf: North_z20_S_5h
        bkg: name Background, obsid None, spatial type CTAIrfBackground, spectral type PowerLaw


# Run the simulation

In [7]:
cpipe.run_sim_obs()
#cpipe.run_sim_quicklook(ShowEvent=True,ShowObsDef=True,ShowSkyModel=True,bkgsubtract='NONE',smoothing_FWHM=0.1*u.deg)

----- ObsID to be observed: ['001']
------- Simulation log -------
=== GApplication ===
 Name ......................: ctobssim
 Version ...................: 1.7.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Sim_ObsDef.xml
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Sim_Model_Unstack.xml
 caldb .....................: prod2
 irf .......................: South_0.5h
 edisp .....................: no
 outevents .................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Events.xml
 prefix ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/TmpEvents
 startindex ................: 1
 seed ......................: 293158
 ra ........................: 83.63
 dec .......................: 22.51
 rad .......................: 5.0
 tmin ......................: 2020-01-01T00:00:00
 tmax ......................: 2020-01-01T00:30:00
 mjdref ....................: 51544.5
 emin ......................: 0.1
 emax 

# Run the analysis

In [8]:
#----- Analysis parameters
cpipe.method_binned = True    # Do a binned analysis
cpipe.method_stack  = True    # Stack the event from different observations in a single analysis?
cpipe.method_ana    = '3D'    # 3D or ONOFF analysis
cpipe.spec_enumbins = 10
cpipe.spec_emin     = 50*u.GeV
cpipe.spec_emax     = 10*u.TeV
cpipe.spec_edisp    = False

# Force the use of the user defined map grid
cpipe.map_UsePtgRef     = False
# Define the map used for the binned analysis
cpipe.map_reso          = 0.05*u.deg # Can be increaded if the code is too slow
cpipe.map_fov           = 8*u.deg    # Can also be reduced (but should increase bkg-cluster degeneracy)
cpipe.map_coord         = copy.deepcopy(cpipe.cluster.coord)
# Define the map used for the template
cpipe.cluster.map_fov   = 2.1*cpipe.cluster.theta_truncation
cpipe.cluster.map_coord = copy.deepcopy(cpipe.cluster.coord)

In [ ]:
cpipe.method_ana    = 'ONOFF'
cpipe.run_ana_dataprep()
cpipe.run_ana_likelihood()


 Starting the data preparation

         This should be used when simulating the data.  
----- ObsID to be analysed: ['001']

=== GApplication ===
 Name ......................: ctselect
 Version ...................: 1.7.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Events.xml
 outobs ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_EventsSelected.xml
 prefix ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Selected
 usepnt ....................: no
 ra ........................: 49.9466666666667
 dec .......................: 41.5130555555556
 rad .......................: 5.65685424949238
 forcesel ..................: yes
 tmin ......................: NONE
 tmax ......................: NONE
 emin ......................: 0.05
 emax ......................: 10
 phase .....................: NONE
 expr ......................: 
 usethres ..................: NONE
 publish ...................: no
 chatter 

In [ ]:
cpipe.method_ana    = '3D'
cpipe.run_ana_dataprep()
cpipe.run_ana_likelihood()

In [ ]:
hdul = fits.open(cpipe.output_dir+'/Ana_Model_Output_Covmat.fits')
hdul.info()
d = hdul[1].data
covariance = d['Covariance'][0,:,:]
v = np.sqrt(np.diag(covariance))
outer_v = np.outer(v, v)
correlation = covariance / outer_v
correlation[covariance == 0] = 0

print(np.array(d['Parameters'][0].split())[[1,5,6,9,10]])
correlation[1,[1,5,6,9,10]]

In [ ]:
cpipe.method_ana = 'ONOFF'
cpipe.run_ana_plot()

In [ ]:
offset     = [0.0,0.5,1.0000,2.0,3.0,4.0,5.0]
TS3D       = [0.0,0.0,1233.3,0.0,0.0,0.0,0.0]
cor3D_bkg1 = [0.0,0.0,0.0638,0.0,0.0,0.0,0.0]
cor3D_bkg2 = [0.0,0.0,0.0292,0.0,0.0,0.0,0.0]
TSOF       = [0.0,0.0,493.98,0.0,0.0,0.0,0.0]

In [ ]:
plt.plot(offset, TS3D)
plt.xlabel('Pointing offset (deg)')
plt.ylabel('TS')